In [1]:
from haystack.components.generators import HuggingFaceTGIGenerator
from getpass import getpass

hf_token = getpass("Enter Hugging Face API key:")

llm = HuggingFaceTGIGenerator("mistralai/Mixtral-8x7B-Instruct-v0.1", token=hf_token)
llm.warm_up()

In [2]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

s3.download_file('core-engineering', 'public/advent/audio.mp3', 'audio.mp3')

In [3]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.audio import LocalWhisperTranscriber

prompt_template = """
You are Santa Clause and need to re-write the following single document, so that it sounds more like Santa. Ho ho ho!
{% for doc in documents %}
document: {{ doc.content }} \n
{% endfor %};
Document in Santa style:
"""
pipeline = Pipeline()
pipeline.add_component(instance=LocalWhisperTranscriber(), name="transcriber")
pipeline.add_component(instance=PromptBuilder(prompt_template), name="prompt_builder")
pipeline.add_component(instance=llm, name="llm")

pipeline.connect("transcriber.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder.prompt", "llm")

In [5]:
result = pipeline.run({"transcriber": {"audio_files": ["audio.mp3"]}})

print(result["llm"]["replies"][0])

ValueError: Missing input: transcriber.sources

This pipeline expects the following inputs:
- transcriber:
    - sources: List[Union[str, Path, ByteStream]]
    - whisper_params: Optional[Dict[str, Any]]
- llm:
    - generation_kwargs: Optional[Dict[str, Any]]
